# Введение в регулярные выражение в python

Для тех, кто в танке) Работа с регулярными выражениями в питоне осуществляется с помощью модуля re. Регулярные выражения служат, как правило, двум целям. Первое. Проверить, что строка удовлетворяет заданному шаблону. Второе. Найти все вхождения шаблона в строку. За первое отвечает match, за второе - search или find. Еще есть опция замены одного шаблона на строку или другой шаблон. За это отвечает sub и subn. Писанные сущности match, search, sub, subn могут быть как функциями модуля re, так и методами скомпилированного  регулярного выражения. Но об этом подробно далее. 

# Основы

In [4]:
import re

Рассматривать примеры пока будем на основании функции match. Функция служит для проверки того, что строка соответствует заданному шаблону. Пример. Пусть есть шаблон 

In [5]:
pattern = r'abc|cde'

Ему удовлетворяют все строки, которые начинаются на abc или на cde. Давайте это проверим. Для удобства создадим функцию check_match для проверки того, описывает ли заданное регулярное выражение заданную строку, или нет.

In [6]:
def check_match(pattern, line, flags=0):
    m = re.match(pattern, line, flags)
    if not m:
        print '"' + line + '"' + " does not match pattern " + '"' + pattern + '"'
        return
    print '"' + line + '"' + " matches patern " + '"' + pattern + '"'
    print '"' + m.group() + '"' + " provides this match"
    print ""

In [7]:
lines = [
    'a',
    'abc', 
    'abcdelkj',
    'cde',
    ' 123 sd'
]
for line in lines:
    check_match(pattern, line)

"a" does not match pattern "abc|cde"
"abc" matches patern "abc|cde"
"abc" provides this match

"abcdelkj" matches patern "abc|cde"
"abc" provides this match

"cde" matches patern "abc|cde"
"cde" provides this match

" 123 sd" does not match pattern "abc|cde"


Что тут нужно заметить. Первое. Если начало строки удовлетворяет шаблону, то вся строка удовлетворяет шаблону. Второе. Если строка не сооветствует шаблону (не матчится), то re.match возвращает None. Третье. Если строка шаблону удовлетворяет, то можно увидеть, по какому набору символов удалось сопоставить (заматчить) шаблон и строку. Для этого достаточно вызвать метод group() у объекта, который возвращает функция re.match().

# Синтаксис регулярных выражений. 

Простейший случай мы разобрали выше. Подряд идущая группа символов описывает точно то, с чем должно совпадать начало строки. Знак вертикальной черты "|", также называемый оператором pipe или конвеером, означает в регулярных выражениях логическое ИЛИ.

Символ точки "." означает любой символ, кроме знака перевода строки "\n". С помощью флагов компиляции можно указать, чтобы точка обозначала вообще любой символ, даже символ перевода строки, но об этом чуть позже. Пример.

In [8]:
pi_pattern = '3.14' # здесь точка означает любой символ, кроме символа перевода строки
exact_pi_pattern = '3\.14' # здесь в шаблоне регулярного выражения \. означает десятичную точку

check_match(pi_pattern, '3.14')
check_match(exact_pi_pattern, '3.14')
check_match(pi_pattern, '3014')
check_match(exact_pi_pattern, '3014')

"3.14" matches patern "3.14"
"3.14" provides this match

"3.14" matches patern "3\.14"
"3.14" provides this match

"3014" matches patern "3.14"
"3014" provides this match

"3014" does not match pattern "3\.14"


Квадратные скобки позволяют задвать набор символов, которые должны стоять на заданном месте. Например регулярное выражение [ab][de] обозначает строки, которые начинаются на ad, или на ae, или на bd, или на be

In [11]:
pattern = '[cr][23][dp][o2]'
check_match(pattern, 'r2d2')
check_match(pattern, 'c3po')
check_match(pattern, 'ololo')

"r2d2" matches patern "[cr][23][dp][o2]"
"r2d2" provides this match

"c3po" matches patern "[cr][23][dp][o2]"
"c3po" provides this match

"ololo" does not match pattern "[cr][23][dp][o2]"


Указав в начале квадратных скобок знак домика "^", можно поменять смысл квадратных скобок на противоположный - теперь они будут обозначать любой знак, который не встречается в квадратных скобках после знака домика. 

In [12]:
pattern = '[^cr][^23][^dp][^o2]'
check_match(pattern, 'r2d2')
check_match(pattern, 'c3po')
check_match(pattern, 'ololo')

"r2d2" does not match pattern "[^cr][^23][^dp][^o2]"
"c3po" does not match pattern "[^cr][^23][^dp][^o2]"
"ololo" matches patern "[^cr][^23][^dp][^o2]"
"olol" provides this match



# Диапазоны символов (range)

В квадратных скобках можно указывать не только наборы символов, но и диапазоны. Например, мы хотим проверить, что строка начинается с четырех цифр, каждая из которых - от двух до семи (уж не знаю, за чем нам понадобились таки странные числа, но все же давайте напряжем воображение). Вместо того, чтобы писать 4 раза [234567], можно 4 раза написать [2-7]

In [13]:
pattern = '[2-7][2-7][2-7][2-7]'
check_match(pattern, 'r2d2')
check_match(pattern, 'c3po')
check_match(pattern, '2534 sdf')

"r2d2" does not match pattern "[2-7][2-7][2-7][2-7]"
"c3po" does not match pattern "[2-7][2-7][2-7][2-7]"
"2534 sdf" matches patern "[2-7][2-7][2-7][2-7]"
"2534" provides this match



# Символы повторения

Согласитесь, что странно 4 раза в примере выше писать один и тот же код "[2-7]". И действительно, есть куча способов сказать, что мы не один раз хотим увидеть цифру от двух до семи, а несколько. Для этого есть специальные символы повторения. Символ * в регулярном выражении означает, что символ может встречаться ноль или более раз, символ + означает, что символ может встречаться один или более раз, запись {4} означает, что символ или группа символов встречается 4 раза. Четверку можно заменить на любое натуральное число. Примеры в студию

In [14]:
check_match('[2-7]*', ' 2345 lkj')
check_match('[2-7]+', 'c3po')
check_match('[2-7]{4}', '2534 sdf')
check_match('[2-7]{5}', '2534 sdf')

" 2345 lkj" matches patern "[2-7]*"
"" provides this match

"c3po" does not match pattern "[2-7]+"
"2534 sdf" matches patern "[2-7]{4}"
"2534" provides this match

"2534 sdf" does not match pattern "[2-7]{5}"


# Специальные символы

Для обозначения цифр, знаков пробелов, цифро-буквенных символов применяются специальные символы.

\d обозначает любую из цифр. Эквивалентен [0-9]

\D обозначает любой символ, кроме цифры. Эквивалентен [^0-9]

\w обозначает любой алфавитно-цифровой символ. Если быть совсем точным, то он эквивалентен [0-9a-zA-Z_]

\W обозначает любой символ, который не попадает под описание  \w из прошлой строк 

\s обозначает любой пробельный символ, например пробел, перевод строки, табуляция и прочее. Эквивалентен [\n\t\r\v\f]

\S обозначет любой символ, не попадающий под описание \s

^ обозначает начало строки

$ обозначает конец строки

\b обозначает границу слова - не важно, начало или конец

\N, где N - это натуральное число, обозначает номер группы. Об этом далее.

Примеры с началом и концом строк, пробелами

In [15]:
patterns = [
    r'^The',
    r'\bThe',
    r'\Bthe',
    r'(\w+\s+){2}\w*the[\w\s]*' # убранное в скобки выражение называется группой, {2} означает, что 
                                # выражение перед {2} должно встретиться два раза. Вместо 2 может быть
                                # любое натуральное число. Можно также здавать не точное число раз, а диапазон
                                # {2,5} говорит о том, что символ или группа перед {2,5} должны встретиться
                                # не меньше двух и не более 5 раз
]
line = 'The best inthe world Hello!'
for p in patterns:
    check_match(p, line)

"The best inthe world Hello!" matches patern "^The"
"The" provides this match

"The best inthe world Hello!" matches patern "\bThe"
"The" provides this match

"The best inthe world Hello!" does not match pattern "\Bthe"
"The best inthe world Hello!" matches patern "(\w+\s+){2}\w*the[\w\s]*"
"The best inthe world Hello" provides this match



# Подробнее про функции и методы классов re

До сих пор я упорно скрывал синтаксис команд регулярных выражений, завернув их в функцию check_match.

Пришло время рассказать подробнее.

В простейшем случае проверка матчинга строки шаблону происходит так

In [16]:
line = 'olololosl'
pattern = r'\w+'

m = re.match(pattern, line)
if m:
    print m.group()

olololosl


re.match возвращает объект регулярного выражения. Если заматчить строку шаблону не получилось, то вернется None, поэтому, чтобы не выкидывались исключения, обычно проверяют, что вернулось - для этого в данном примере используется if.

метод group объекта регулярного выражения возвращает стоку, которую удалось сопоставить регулярному выражению.

Все, что в регулярном выражении заключено в круглые скобки, считается так называемой группой, и к элементам группы можно обращаться через метод group, в случае удачного матча строки и регулярного выражения

In [17]:
line = 'abc-123'
pattern = r'(\w+)-(\d+)'

m = re.match(pattern, line)
if m:
    print m.group(1)
    print m.group(2)
    print m.groups() # еще есть метод groups, который сразу вернет содержимое групп в виде кортежа

abc
123
('abc', '123')


Согласитесь, полезная штука. Позволит доставать части регулярно выражения задешево. Без дополнительных плат на то, чтобы разделять то, что нашлось с помощью регулярного выражения.

Но с помощью регулярных выражений можно не только проверять, подходит ли строка под шаблон, но и находить шаблоны в строке. Для этого используется функция re.search или одноименный метод объекта скомпилированного регулярного выражения (у объектов скомпилированных регулярных выражений есть и метод match тоже; об этом чуть далее будет отдельно). С помощью этого метода можно проверить вхождение шаблона в строку, шаблон может входить в строку в любом месте, не только в начале

Простейший пример

In [14]:
line = 'abcdef 098'
patterns = [
    r'\w',
    r'\w+',
    r'\b09'
]
for p in patterns:
    s = re.search(p, line)
    print s.group()

a
abcdef
09


Также можно не только проверить факт вхождения шаблона в строку, но и найти все непересекающиеся вхождения этого шаблона в строку. Делается это с помощью функции findall. Для примера давайте найдем все слова в строке

In [15]:
line = 'this car is very good'
pattern = r'\w+'
result = re.findall(pattern, line)

In [16]:
print result

['this', 'car', 'is', 'very', 'good']


Но мы-то с вами продвинутые пользователи, и знаем, что если строка окажется очень большой, то такая функция вернет очень длинный лист. Поэтому эта функция имеет эквивалентную ей, которая возвращает не лист с совпадениями, а итератор на такой лист. Функция называется finditer. Давайте рассмотрим на примере.

In [17]:
line = 'this car is very good'
pattern = r'\w+'
result_iter = re.finditer(pattern, line)

In [18]:
result_iter.next().group()

'this'

In [19]:
result_iter.next().group()

'car'

In [20]:
result_iter.next().group()

'is'

In [21]:
result_iter.next().group()

'very'

In [22]:
result_iter.next().group()

'good'

In [23]:
result_iter.next().group()

StopIteration: 

Все отработало, как положено уважающему себя итератору - при вызове метода next() он возвращает следующий удачный результат поиска. Когда результаты кончаются, он выкидывает исключение StopIteration, как это делают все итераторы. 

Еще осталось несколько функций, и на этом зевершим раздел. Регулярные выражения позволяют делать замену части строки, которая удовлетворяет регулярному выражению. Делается с помощью функции sub или subn. Вторая отличается от первой тем, что возвращает и еще число раз, которое оно заменило регулярное выражение на то, что мы указали. Заменять можно как на что-то фиксированное, так и переставлять разные части строки, совпадающие с разными группами, созданными в шаблоне регулярного выражения. Звучит, наверно, диковато, поэтому давайте к примерам. Сначала простой пример. Хотим заменить все подряд идущие цифры на XXXX.

In [18]:
line = 'It is 203948 good 2398 for 0239 us.'
pattern = r'\d+'
print re.sub(pattern, 'XXXX', line)

It is XXXX good XXXX for XXXX us.


Можно еще вызвать subn - тогда она вернет кортеж из новой строки и числа замен в строке

In [19]:
line = 'It is 203948 good 2398 for 0239 us.'
pattern = r'\d+'
print re.subn(pattern, 'XXXX', line)

('It is XXXX good XXXX for XXXX us.', 3)


Согласитесь, полезная штука. Например вы хотите отправить 200 писем, а списко адресатов еще не готов. Просто вместо имени пишете XXXX, а потом в цикле заменяете с помощью регулярного выражения XXXX на имя получателя письма. Кто-то скажет, что это можно было сделать с помощью метода replace класса обычной питоновской строки. Что-ж, верно. Тогда пример посложнее, заодно с группами поработаем. Пусть вам прислали в файле даты в американском формате - ГГГГ-ДД-ММ, то есть сначала написан год, потом день, потом месяц. А вы к такомы не привыкли - неудобно. Надо переставить местами месяцы и дни. Так вот тут нам на помощь и придут группы. К ним не только можно иметь отдельный доступ после того, как строка подошла шаблону после re.match или re.search, но еще и использовать их во втором аргументе функции re.sub, чтобы задать параметры замены. Делается с помощью \N, где N - номер группы. Чтобы стало понятнее - пример.

In [27]:
american_date = '2015-20-11'
date_pattern_with_groups = r'(\d{2}|\d{4})-(\d{1,2})-(\d{1,2})'
# первая группа - год из двух или четырех цифр, потом тире, потом день из одной или двух цифр, потом месяц
# хотим переставить местами месяц и день
replace_pattern = r'\1-\3-\2' # так и говорим - хотим сначала первую группу, потом - третью, потом - первую

Теперь, собвственно, замена

In [28]:
re.sub(date_pattern_with_groups, replace_pattern, american_date)

'2015-11-20'

Вуаля, вы великолепны!

# Про объекты регулярных выражений

Для чего они нужны? Есть же функции re.search, re.match и прочие. Ответ такой. Нужны для ускорения кода. Когда вызывается функция re.search, то, что задано в качестве шаблона компилируется кодом на C в бинарное представление специальными средствами модуля re. Делается это каждый раз, когда вы вызваете функцию. Если вы вызываете ее 100500 раз в цикле, то 100500 раз будет происходить компиляция. Спрашивается - зачем? Можно же один раз скомпилировать и сэеономить кучу времени. Вот для этого и нужны объекты регулярных выражений. Пример

In [25]:
compiled_re = re.compile(r'\d+')  # выражение для поиска групп из хотя бы одной цифры в строке

In [22]:
result = compiled_re.search('laksjdf')
print result

result = compiled_re.search('sldkfj 098')
print result

None


In [24]:
result.group()

'098'

Видно, что можно повторно использовать скомпилированный объект класса регулярного выражения. Можно даже не только для поиска, но и для замены, матчинга и поиска.

In [28]:
# пример с заменой
line = 'ololo 909 ololo 098 ololo'

result = compiled_re.sub('XXXX', line)
print result

# пример с поиском
result = compiled_re.findall(line)
print result

ololo XXXX ololo XXXX ololo
['909', '098']


# Про жадность

Операторы *,+,?,{} работают жадным образом. То есть они пытаются найти строку как можно большей длины, которая удовлетворяет шаблону. Давайте сразу пример. Пусть мы хотим вытащить из строки последовательность цифр, разделенную знаками -. Например, телефонный номер, который записан на конце строки. А в начале строки есть какой-то текст. 

In [29]:
line = ' s;dlfkj s;dflkj s;dlkfj s;ldfkj 111-11-11'

Ну и казалось бы, что может быть легче - сначала пропустим все символы, потом будем искать номер телефона. 

In [35]:
pattern = '.*(\d+-\d+-\d+)'
re.match(pattern, line).group(1)

'1-11-11'

Видно, что первые две единицы отрезало. Это из-за того, что часть .* регулярного выражения нашла последовательность максимальной длины, которая ей удовлетворяет. От этого эффекта можно избавиться, если указать оператору *, что надо использовать его нежадную версию - чтобы он не искал самую длинную подстроку, а нашел самую короткую, которая подходит. Это делается с помощью записи ? после *. Это же относится ко всем операторам - *,+,?,{}

In [36]:
pattern = '.*?(\d+-\d+-\d+)'
re.match(pattern, line).group(1)

'111-11-11'

Вуаля, вы снова великолепны! =)

# На этом хватит, для начала

Для элементарной работы с регулярными выражениями этого хватит. В будущем планирую дописать ноутбуки с примерами  работы с флагами компиляции и с расширенным синтаксисом регулярных выражений. 

In [37]:
# Best regards, steninss